<a href="https://colab.research.google.com/github/massone99/visione_artificiale_colab_notebooks/blob/main/ObjectTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Esercitazione su object tracking**
Nell'esercitazione odierna implementeremo due applicazioni di *object tracking* utilizzando rispettivamente gli algoritmi di *Mean-Shift* e di sottrazione del backgorund tramite *Mixture Of Gaussian*.

# **Operazioni preliminari**
Eseguendo la cella sottostante tutto il materiale necessario per lo svolgimento dell'esercitazione verrà scaricato sulla macchina remota. Alla fine dell'esecuzione selezionare nel menù laterale **File** per verificare che tutto sia stato scaricato correttamente.

In [ ]:
!wget http://bias.csr.unibo.it/VR/Esercitazioni/MaterialeEsObjectTracking.zip

!unzip /content/MaterialeEsObjectTracking.zip

!rm /content/MaterialeEsObjectTracking.zip

# **Import delle librerie**
Per prima cosa è necessario eseguire l'import delle librerie utilizzate durante l'esecitazione.

In [ ]:
import os
import numpy as np
import cv2
import ipywidgets as widgets
import uuid
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from ipywidgets import interact, fixed
from IPython.display import HTML
from base64 import b64encode

# **Functioni di utilità**
Tramite la cella sottostante è possibile definire alcune funzioni di utilità usate durante l'esercitazione:
- **create_mp4_video_from_frames** crea un video e lo salva in formato MP4 partendo da una lista di *frame*;
- **draw_connected_components** visualizza le componenti connesse presenti su un'immagine binaria;
- **draw_detected_objects** visualizza le *bounding-box* degli oggetti individuati su un singolo frame;
- **draw_tracked_objects** visualizza gli oggetti tracciati su un singolo frame;
- **compute_iou** calcola la *intersection over union* tra due *bounding-box*.

In [ ]:
def create_mp4_video_from_frames(frames,fps):
  temp_video_path='tempfile.mp4'
  compressed_path='{}.mp4'.format(str(uuid.uuid4()))

  size=(frames[0].shape[1],frames[0].shape[0])
  out = cv2.VideoWriter(temp_video_path,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

  for i in range(len(frames)):
      out.write(frames[i][...,::-1].copy())
  out.release()

  os.system(f"ffmpeg -i {temp_video_path} -vcodec libx264 {compressed_path}")

  os.remove(temp_video_path)

  return compressed_path

def draw_connected_components(labels):
  label_hue = np.uint8(179*labels/np.max(labels))
  blank_ch = 255*np.ones_like(label_hue)
  labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

  labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

  labeled_img[label_hue==0] = 0

  return labeled_img

def draw_detected_objects(image,detected_bbs,detected_centroids,contours=None,detected_colors=(0,0,255)):
  if contours is not None:
    image_with_detected_objects=cv2.polylines(image.copy(),contours,True,(0,255,0),1)
  else:
    image_with_detected_objects=image.copy()

  for i in range(len(detected_bbs)):
    if type(detected_colors) is list:
      color=detected_colors[i]
    else:
      color=detected_colors

    image_with_detected_objects=cv2.rectangle(image_with_detected_objects,detected_bbs[i][0],detected_bbs[i][1],color,1)
    image_with_detected_objects=cv2.circle(image_with_detected_objects,detected_centroids[i],3,color,-1)

  return image_with_detected_objects

def draw_tracked_objects(image,tracked_bbs,tracks,contours=None,tracked_colors=(0,0,255)):
  if contours is not None:
    image_with_tracked_objects=cv2.polylines(image.copy(),contours,True,(0,255,0),1)
  else:
    image_with_tracked_objects=image.copy()

  for i in range(len(tracked_bbs)):
    if type(tracked_colors) is list:
      color=tracked_colors[i]
    else:
      color=tracked_colors

    image_with_tracked_objects=cv2.rectangle(image_with_tracked_objects,tracked_bbs[i][0],tracked_bbs[i][1],color,1)
    for j in range(len(tracks[i])-1):
      image_with_tracked_objects=cv2.line(image_with_tracked_objects,tracks[i][j],tracks[i][j+1],color,1)

  return image_with_tracked_objects

def compute_iou(bb1, bb2):
  bb1_x1=bb1[0][0]
  bb1_y1=bb1[0][1]
  bb1_x2=bb1[1][0]
  bb1_y2=bb1[1][1]

  bb2_x1=bb2[0][0]
  bb2_y1=bb2[0][1]
  bb2_x2=bb2[1][0]
  bb2_y2=bb2[1][1]

  x_left = max(bb1_x1, bb2_x1)
  y_top = max(bb1_y1, bb2_y1)
  x_right = min(bb1_x2, bb2_x2)
  y_bottom = min(bb1_y2, bb2_y2)

  if x_right < x_left or y_bottom < y_top:
    return 0.0

  intersection_area = (x_right - x_left) * (y_bottom - y_top)

  bb1_area = (bb1_x2 - bb1_x1) * (bb1_y2 - bb1_y1)
  bb2_area = (bb2_x2 - bb2_x1) * (bb2_y2 - bb2_y1)

  iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
  assert iou >= 0.0
  assert iou <= 1.0
  return iou

# **Single object tracking**
L'obiettivo è quello di sviluppare un'applicazione che, utilizzando l'algoritmo *mean-shift*, è in grado di seguire, all'interno di una sequenza video, il movimento di un **singolo** oggetto selezionato **manualmente** dall'utente.

L'algoritmo *mean-shift* è composto da **due** fasi:
1. calcolo delle **feature colore** dell'oggetto selezionato sul primo frame;

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsObjectTracking\MeanShiftFase1.png width="400">

2. per ogni successivo frame, ricerca del **candidato** più **simile** all'interno della **regione di interesse**.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsObjectTracking\MeanShiftFase2.png width="800">

## **Caricare un video tramite OpenCV**
OpenCV mette a disposizione la classe **VideoCapture** che permette di gestire video:
- caricati da file;
- rappresentati da sequenze di immagini;
- acquisiti direttamente da una camera.

Per poter creare un'istanza della classe a partire da un file è sufficiente richiamare il costruttore passando come parametro il percorso del file.

In [ ]:
video_capture = cv2.VideoCapture('/content/mouthwash.avi')

Il metodo **get(...)** della classe **VideoCapture** permette di accedere alle proprietà del video caricato.

In [ ]:
frame_count=int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
video_width=int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height=int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

print('Numero totale di frame:',frame_count)
print('Dimensione video:',video_width,'x',video_height)

## **Fattore di resize**
Nel caso che le dimensioni del video siano elevate, per ridurre i tempi di calcolo può essere utile ridimensionare i singoli frame prima di elaborarli. Per fare ciò, impostare nella cella sottostante il fattore di resize (*resize_factor*) desiderato (1=no resize).

In [ ]:
resize_factor=0.5

resized_width=int(resize_factor*video_width)
resized_height=int(resize_factor*video_height)

print('Dimensione resize:',resized_width,'x',resized_height)

## **Lettura di un singolo frame**
Tramite il metodo **read(...)** è possibile ottenere il prossimo frame del video. Oltre al frame, il metodo restituisce un booleano che sarà impostato a *False* nel caso in cui non sia disponibile un nuovo frame (ad esempio perché la sequenza video è terminata). In tal caso verrà restituito un frame vuoto.

In [ ]:
ret,first_frame = video_capture.read()

if resize_factor!=1:
  first_frame=cv2.resize(first_frame,(resized_width,resized_height),cv2.INTER_LINEAR)

print('Dimensione:',first_frame.shape)
print('Formato:',first_frame.dtype)

cv2_imshow(first_frame)

## **Fase 1**
L'obiettivo della fase 1 è calcolare le feature colore dell'oggetto che si vuole tracciare.

### **Selezione della regione di interesse**
Tramite la cella sottostante è possibile selezionare la regione contenente l'oggetto da seguire.

Per poter tracciare gli spostamenti della bottiglia si consiglia di selezionare la regione contenente l'etichetta.

In [ ]:
w_h_range=widgets.IntRangeSlider(description='Range Y:',min=0,max=first_frame.shape[0]-1,value=(0,first_frame.shape[0]-1), step=1,continuous_update=False)
w_w_range=widgets.IntRangeSlider(description='Range X:',min=0,max=first_frame.shape[1]-1,value=(0,first_frame.shape[1]-1), step=1,continuous_update=False)

@interact(frame=fixed(first_frame),h_range=w_h_range,w_range=w_w_range)
def interactive_roi_selection(frame,h_range,w_range):
  frame_with_roi=cv2.rectangle(frame.copy(),(w_range[0],h_range[0]),(w_range[1],h_range[1]),(0,0,255),1)
  cv2_imshow(frame_with_roi)

Selezionata la regione di interesse (*RoI*), la rappresenteremo sotto forma di tupla: $(\text{min}_X,\text{min}_Y,W,H)$.

In [ ]:
first_frame_roi_rect = (w_w_range.value[0],w_h_range.value[0],w_w_range.value[1]-w_w_range.value[0]+1,w_h_range.value[1]-w_h_range.value[0]+1)

print('Regione di interesse:',first_frame_roi_rect)

Utilizzando lo *slicing* degli **ndarray** e il rettangolo appena creato, ritagliare la *RoI* dal frame iniziale.

In [ ]:
roi=#...

print('Dimensione:',roi.shape)

cv2_imshow(roi)

### **Calcolo delle feature colore**
Come *feature* utilizzeremo l'**istogramma colore** calcolato sul canale H (tinta) nello spazio colore HSV. L'istogramma sarà calcolato sul canale H per ottenere feature colore più robuste rispetto a variazioni di luminosità e contrasto.

Come prima cosa si dovrà convertire la regione di interesse dallo spazio colore BGR a quello HSV. Per farlo sarà sufficiente utilizzare la funzione **cvtColor(...)** di OpenCV.

In [ ]:
hsv_roi=#...

print('Dimensione:',hsv_roi.shape)

cv2_imshow(hsv_roi)

Per aumentare la robustezza delle feature estratte, può essere utile calcolarle non sull'intera regione di interesse ma solo sui pixel che presentano i colori più caratteristici.

La funzione **calcHist(...)** di OpenCV (che utilizzeremo per calcolare l'istogramma) può ricevere come parametro di input una maschera binaria che rappresenta i pixel su cui operare. Nella cella seguente si dovranno scegliere gli opportuni range di H, S e V per creare tale maschera avendo cura che solo la parte di interesse rimanga visibile mentre tutto ciò che non è influente venga mascherato.

In [ ]:
w_h_range=widgets.IntRangeSlider(description='Range H:',min=0,max=255,value=(0,255), step=1,continuous_update=False)
w_s_range=widgets.IntRangeSlider(description='Range S:',min=0,max=255,value=(0,255), step=1,continuous_update=False)
w_v_range=widgets.IntRangeSlider(description='Range V:',min=0,max=255,value=(0,255), step=1,continuous_update=False)

@interact(hsv_roi=fixed(hsv_roi),roi=fixed(roi),h_range=w_h_range,s_range=w_s_range,v_range=w_v_range)
def interactive_masked_roi(hsv_roi,roi,h_range,s_range,v_range):
  mask = cv2.inRange(hsv_roi, (h_range[0],s_range[0],v_range[0]), (h_range[1],s_range[1],v_range[1]))
  cv2_imshow(cv2.bitwise_and(roi,roi, mask=mask))

Creare la maschera binaria utilizzando la funzione **inRange(...)** passandogli in input i range individuati nella cella precedente.

In [ ]:
h_min_value=w_h_range.value[0]
h_max_value=w_h_range.value[1]
s_min_value=w_s_range.value[0]
s_max_value=w_s_range.value[1]
v_min_value=w_v_range.value[0]
v_max_value=w_v_range.value[1]

roi_mask=#...

cv2_imshow(roi_mask)

La funzione **calcHist(...)** utilizzata per calcolare l'istogramma riceve in input diversi parametri tra cui:
- il numero di *bin* dell'istogramma;
- il range di valori da considerare sul canale selezionato.

Utilizzare la cella sottostante per selezionare i valori opportuni per tali parametri.

In [ ]:
w_bin_count=widgets.IntSlider(min=10, max=256, step=1, value=256,continuous_update=False)
w_h_range=widgets.IntRangeSlider(description='Range H:',min=0,max=255,value=(0,255), step=1,continuous_update=False)

@interact(image=fixed(hsv_roi),channel=fixed(0),mask=fixed(roi_mask),bin_count=w_bin_count,h_range=w_h_range)
def interactive_hist(image,channel,mask,bin_count,h_range):
  hist = cv2.calcHist([image],[channel],mask,[bin_count],[h_range[0],h_range[1]])
  plt.plot(hist)
  plt.xlim([0,bin_count])
  plt.show()

Richiamare la funzione **calcHist(...)** per calcolare l'istogramma sul canale H della regione di interesse (*hsv_roi*) utilizzando i parametri opportuni (*mask*, *bin_count* e *h_range*).

In [ ]:
bin_count=w_bin_count.value
h_range=[w_h_range.value[0],w_h_range.value[1]]

roi_h_hist=#...

plt.plot(roi_h_hist)
plt.xlim([0,bin_count])
plt.show()

L'altezza dell'istogramma calcolato dipende dal numero di pixel su cui è stato calcolato. Pertanto è opportuno normalizzarlo per renderlo indipendente dalla dimensione della regione di interesse.

Il codice contenuto nella cella sottostante utilizza la funzione **normalize(...)** di OpenCV per normalizzare l'istogramma appena calcolato così che presenti un'altezza massima di 255 indipendentemente dal numero di pixel su cui è stato calcolato.

In [ ]:
cv2.normalize(roi_h_hist,roi_h_hist,0,255,cv2.NORM_MINMAX)

plt.plot(roi_h_hist)
plt.xlim([0,bin_count])
plt.show()

## **Fase 2**
L'obiettivo della fase 2 è ricercare in un frame la posizione del candidato più simile all'oggetto da tracciare selezionato nella fase 1 (in termini di feature colore).


### **Calcolo della mappa di probabilità**
Dato un  frame, per prima cosa si calcola la mappa di probabilità che indica la probabilità che ogni pixel appartenga a una regione il cui istogramma colore è simile a quello dell'oggetto *target*.

La cella sottostante carica un nuovo frame.

In [ ]:
_,frame = video_capture.read()

if resize_factor!=1:
  frame=cv2.resize(frame,(resized_width,resized_height),cv2.INTER_LINEAR)

cv2_imshow(frame)

Convertiamo il frame corrente nello spazio HSV tramite la funzione **cvtColor(...)**.

In [ ]:
hsv_frame=#...

cv2_imshow(hsv_frame)

Richiamando la funzione **calcBackProject(...)** di OpenCV si può calcolare la mappa di probabilità dati:
- l'immagine di input (*hsv_frame*);
- il canale su cui calcolarla (H=0);
- l'istogramma dell'oggetto target (*roi_h_hist*);
- il range di valori da considerare durante il calcolo dell'istogramma (*h_range*);
- la scala con cui calcolare la mappa di probabilità (1).

In [ ]:
back_proj=#...

cv2_imshow(back_proj)

Per ridurre la possibilità di incorrere in falsi positivi può essere utile considerare solo quei pixel che presentano colori simili a quelli che caratterizzano l'oggetto target.

Per creare la maschera binaria dei pixel validi, è sufficiente utilizzare la funzione **inRange(...)** come abbiamo già fatto nella fase 1.

In [ ]:
frame_mask=#...

cv2_imshow(frame_mask)

Utilizzando la funzione **bitwise_and(...)** è possibile annullare tutti i valori della mappa di probabilità che non corrispondono a pixel validi.

In [ ]:
back_proj=#...

cv2_imshow(back_proj)

### **Individuazione del candidato più simile**
Ora non resta che individuare l'oggetto più simile a quello target sulla base della mappa di probabilità appena calcolata.

La funzione **meanShift(...)** di OpenCV restituisce la *bounding box* che contiene l'oggetto più simile a quello target, cercandolo vicino a dove si trovava nel frame precedente. I parametri di input sono:
- la mappa di probabilità (*back_proj*);
- la *bounding box* in cui si trovata l'oggetto target nel frame precedente;
- il criterio di stop con cui terminare la ricerca (*term_crit*).

In [ ]:
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

_, roi_rect=#...

La cella sottostante permette di visualizzare sulla mappa di probabilità la bounding box iniziale dell'oggetto target (in rosso) e la bounding box individuata sul frame corrente (in blu).

In [ ]:
back_proj_with_roi=cv2.rectangle(cv2.cvtColor(back_proj.copy(), cv2.COLOR_GRAY2BGR),(first_frame_roi_rect[0],first_frame_roi_rect[1]),(first_frame_roi_rect[0]+first_frame_roi_rect[2],first_frame_roi_rect[1]+first_frame_roi_rect[3]),(0,0,255),1)
back_proj_with_roi=cv2.rectangle(back_proj_with_roi,(roi_rect[0],roi_rect[1]),(roi_rect[0]+roi_rect[2],roi_rect[1]+roi_rect[3]),(255,0,0),1)
cv2_imshow(back_proj_with_roi)

### **Tracciamento sull'intera sequenza video**
Completare il codice sottostante per tracciare l'oggetto selezionato sull'intera sequenza video. In particolare:

1. leggere un nuovo frame;
2. ridimensionare il frame in base al *resize_factor*;
3. convertire il frame nello spazio colore HSV;
4. calcolare la mappa di probabilità;
5. creare la maschera binaria dei pixel che presentano colori simili a quelli che caratterizzano l'oggetto target;
6. mascherare la mappa di probabilità utilizzando la maschera creata al punto precedente;
7. utilizzare l'algoritmo *mean-shift* per individuare la posizione corrente dell'oggetto.

Il metodo **set(...)** della classe **VideoCapture** permette di impostare le proprietà di una istanza video. Nel nostro caso viene utilizzato per riportare all'inizio la posizione di lettura della sequenza video.

In [ ]:
tracking_rgb_frames=[]
tracking_back_projections=[]

video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

roi_rect=first_frame_roi_rect
while True:
    #1
    ret ,frame=#...

    if ret:
        #2
        if resize_factor!=1:
          frame=#...

        #3
        hsv_frame=#...

        #4
        back_proj=#...

        #5
        frame_mask=#...

        #6
        back_proj=#...

        #7
        _, roi_rect=#...

        frame_with_roi = cv2.rectangle(frame, (roi_rect[0],roi_rect[1]), (roi_rect[0]+roi_rect[2],roi_rect[1]+roi_rect[3]), (0,0,255),1)
        tracking_rgb_frames.append(cv2.cvtColor(frame_with_roi, cv2.COLOR_BGR2RGB))

        back_proj_with_roi= cv2.rectangle(cv2.cvtColor(back_proj,cv2.COLOR_GRAY2RGB), (roi_rect[0],roi_rect[1]), (roi_rect[0]+roi_rect[2],roi_rect[1]+roi_rect[3]), (255,0,0),1)
        tracking_back_projections.append(back_proj_with_roi)
    else:
        break

print('Numero di frame elaborati:',len(tracking_rgb_frames))

Il codice seguente crea un video in cui è riportato il risultato dell'applicazione implementata.

In [ ]:
tracking_combined=[]
for i in range(len(tracking_rgb_frames)):
  tracking_combined.append(np.concatenate((tracking_rgb_frames[i], tracking_back_projections[i]), axis=1))

vide_file_name=create_mp4_video_from_frames(tracking_combined,30)

mp4 = open(vide_file_name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(""" <video controls><source src="%s" type="video/mp4"></video>""" % data_url)

## Esercizio
Testare l'applicazione appena implementata sul video "nascar.mp4".

# **Multiple object tracking**
L'obiettivo è quello di sviluppare un'applicazione che, utilizzando l'algoritmo *Mixture Of Gaussian* per effettuare la sottrazione del background, è in grado di seguire oggetti **multipli** all'interno di una sequenza video.

L'algoritmo è composto da **tre** fasi:

1. **stima** del **background** calcolato su una porzione iniziale del flusso video;
2. **individuazione** degli **oggetti** in movimento rispetto al background;
3. **abbinamento** degli **oggetti** individuati con quelli individuati nel frame **precedente**.

## **Caricamento del video**
La cella sottostante crea un'istanza della classe **VideoCapture** caricando un file da disco.

In [ ]:
video_capture = cv2.VideoCapture('/content/M6 Motorway Traffic.mp4')

frame_count=int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
video_width=int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height=int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

print('Numero totale di frame:',frame_count)
print('Dimensione video:',video_width,'x',video_height)

## **Fattore di resize**
Nel caso che le dimensioni del video siano elevate, per ridurre i tempi di calcolo può essere utile ridimensionare i singoli frame prima di elaborarli. Per fare ciò, impostare nella cella sottostante il fattore di resize (*resize_factor*) desiderato (1=no resize).

In [ ]:
resize_factor=0.5

resized_width=int(resize_factor*video_width)
resized_height=int(resize_factor*video_height)

print('Dimensione resize:',resized_width,'x',resized_height)

## **Lettura primo frame**
Eseguendo la cella seguente verrà letto il primo frame della sequenza video caricata. Nel caso che *resize_factor* sia diverso da 1, il frame verrà ridimensionato in maniera opportuna.

In [ ]:
_,first_frame = video_capture.read()

if resize_factor!=1:
  first_frame=cv2.resize(first_frame,(resized_width,resized_height),cv2.INTER_LINEAR)

print('Dimensione:',first_frame.shape)
print('Formato:',first_frame.dtype)

cv2_imshow(first_frame)

## **Regione di interesse**
In questo tipo di applicazioni può essere utile definire la regione di interesse su cui l'algoritmo dovrà lavorare così da ridurre la presenza di falsi positivi in zone ininfluenti.

La cella seguente definisce la regione di interesse come un insieme di poligoni e la visualizza sul frame corrente.

In [ ]:
left_side_contour=np.array([[0,719],[0,510], [515,280],[625,280],[490,719] ])
right_side_contour=np.array([[790,719],[655,280],[750,280], [1279,560],[1279,719] ])

if resize_factor!=1:
  for i in range(len(left_side_contour)):
      left_side_contour[i]=left_side_contour[i]*resize_factor
  for i in range(len(right_side_contour)):
      right_side_contour[i]=right_side_contour[i]*resize_factor

frame_with_roi=cv2.polylines(first_frame,[left_side_contour,right_side_contour],True,(0,255,0),1)

cv2_imshow(frame_with_roi)

Per poter utilizzare in maniera efficiente la regione di interesse, è bene rappresentarla con una maschera binaria.

In [ ]:
roi_mask=np.zeros((resized_height,resized_width),np.uint8)
cv2.fillPoly(roi_mask, pts = [left_side_contour,right_side_contour], color =(255,255,255))

cv2_imshow(roi_mask)

## **Fase 1**
L'obiettivo della fase 1 è la stima del background partendo dalla porzione iniziale del flusso video.

La cella seguente calcola il numero di frame da utilizzare nella stima del background a partire da una percentuale di input.

In [ ]:
bg_est_frame_perc = 0.01

train_frame_count=int(frame_count*bg_est_frame_perc)

print('Numero di frame utilizzati per la stima del BG:',train_frame_count)

### **Creazione della *Mixture Of Gaussian***
La funzione **createBackgroundSubtractorMOG2(...)** di OpenCV crea un'istanza della classe **BackgroundSubtractorMOG2** che implementa un algoritmo di sottrazione del background basato su *Mixture Of Gaussian*.

In [ ]:
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

### **Stima background iniziale**
Completare il codice sottostante per stimare il background iniziale. In particolare:

1. leggere un nuovo frame;
2. ridimensionare il frame in base al *resize_factor*;
3. aggiornare il background tramite il metodo **apply(...)** della classe **BackgroundSubtractorMOG2**;
4. memorizzare l'immagine del background attuale nella variabile *current_bg* tramite il metodo **getBackgroundImage(...)**.

Il metodo **apply(...)**, presi in input un frame e il *learning rate*, sottrae il background dal frame di input restituendo la maschera di foreground dopodiché aggiorna il background. Visto che in questa fase vogliamo stimare il backgorund iniziale, non sarà necessario memorizzare la maschera di foreground restituita. Infine, se il *learning rate* non viene passato al metodo, l'algoritmo calcola internamente il *learning rate* più opportuno.

Attenzione, se si vuole provare a stimare il background al variare del numero di frame e del *learning rate* utilizzati, è necessario ogni volta re-inizializzare la variabile *bg_subtractor*.

In [ ]:
video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

estimated_bg=[]
k=0
while k<train_frame_count:
  #1
  ret, frame=#...
  if ret==False:
    break

  #2
  if resize_factor!=1:
    frame=#...

  #3
  #...

  #4
  current_bg=#...

  estimated_bg.append(cv2.cvtColor(current_bg, cv2.COLOR_BGR2RGB))

  k+=1

Il codice seguente crea un video in cui è mostrata l'evoluzione del background stimato dalla *Mixture Of Gaussian*.

In [ ]:
vide_file_name=create_mp4_video_from_frames(estimated_bg,120)

mp4 = open(vide_file_name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(""" <video controls><source src="%s" type="video/mp4"></video>""" % data_url)

## **Fase 2**
L'obiettivo della fase 2 è, dato un frame, individuare la posizione degli oggetti in movimento.

La cella sottostante carica un nuovo frame.

In [ ]:
_ ,frame = video_capture.read()

if resize_factor!=1:
  frame=cv2.resize(frame,(resized_width,resized_height),cv2.INTER_LINEAR)

print('Dimensione:',first_frame.shape)
print('Formato:',first_frame.dtype)

cv2_imshow(frame)

### **Segmentazione del foreground**
Utilizzare il metodo **apply(...)** per sottrarre il background dal frame corrente e ottenere la maschera di *foreground*.

Si noti che, se il *learning_rate* è diverso da zero, il background verrà mantenuto aggiornato.

In [ ]:
fg=#...

print('Dimensione:',fg.shape)
print('Formato:',fg.dtype)

cv2_imshow(fg)

### **Post-processing del foreground**
La maschera di foreground così ottenuta necessità di alcune operazioni di post-processing per migliorarne la qualità e rendere più robusta l'individuazione degli oggetti in movimento.  

I pixel dell'immagine restituita dal metodo **apply(...)** della classe **BackgroundSubtractorMOG2** presentano tre possibili valori di intensità corrispondenti a tre diverse classi:

- background (0);
- shadows (127);
- foreground (255).

È possibile ottenere una maschera binaria eliminando le ombre (*shadows*) tramite la funzione **threshold(...)** con soglia di binarizzazione maggiore di 127.

In [ ]:
bin_thr=128

_,fg=#...

print('Dimensione:',fg.shape)
print('Formato:',fg.dtype)

cv2_imshow(fg)

Solo gli oggetti in movimento all'interno della regione di interesse sono significativi ai fini dell'applicazione.

Richiamare la funzione **bitwise_and(...)** di OpenCV per eseguire l'AND logico tra i rispettivi pixel delle maschere binarie di *foregorund* e della regione di interesse.

In [ ]:
fg=#...

print('Dimensione:',fg.shape)
print('Formato:',fg.dtype)

cv2_imshow(fg)

Per eliminare piccole zone di rumore e rendere più omogenee le aree di foreground possono essere utilizzati gli operatori morfologici.

Il codice sottostante crea un filtro (o *kernel*) di forma circolare con dimensione *morph_kernel_size*$\times$*morph_kernel_size*.

In [ ]:
morph_kernel_size=5

half_morph_kernel_size=int(morph_kernel_size/2)

morph_kernel=np.zeros((morph_kernel_size, morph_kernel_size), dtype = "uint8")
cv2.circle(morph_kernel, (half_morph_kernel_size, half_morph_kernel_size), half_morph_kernel_size, 255, -1)

print('Dimensione:',morph_kernel.shape)
print('Formato:',morph_kernel.dtype)

print(morph_kernel)

Chiudere i piccoli buchi presenti negli oggetti applicando l'operatore morfologico di **chiusura** alla maschera di foreground utilizzando il kernel creato in precedenza.

Per farlo, richiamare la funzione **morphologyEx(...)** di OpenCV con il parametro *op* uguale a cv2.MORPH_CLOSE.

In [ ]:
fg=#...

print('Dimensione:',fg.shape)
print('Formato:',fg.dtype)

cv2_imshow(fg)

Rimuovere piccoli oggetti (rumore) utilizzando l'operatore morfologico di **apertura** usando lo stesso kernel utilizzato per l'operazione precedente.

La funzione **morphologyEx(...)** può essere utilizzata per eseguire la chiusura impostando il parametro *op* uguale a cv2.MORPH_OPEN.

In [ ]:
fg=#...

print('Dimensione:',fg.shape)
print('Formato:',fg.dtype)

cv2_imshow(fg)

Infine applicare un operatore di **dilatazione** per chiudere (o ridurre) i buchi rimasti oltre che per ingrandire leggermente gli oggetti di foreground.

La funzione **morphologyEx(...)** può essere utilizzata per eseguire la dilatazione impostando il parametro *op* uguale a cv2.MORPH_DILATE.

In [ ]:
fg=#...

print('Dimensione:',fg.shape)
print('Formato:',fg.dtype)

cv2_imshow(fg)

### **Individuazione dei possibili candidati**
Per individuare tutti i possibili candidati presenti nella maschera di foreground ottenuta dopo la fase di post-processing si usi l'algoritmo delle componenti connesse.

Richiamare la funzione di OpenCV **connectedComponentsWithStats(...)** passandogli in input la maschera di foregound. La funzione restituirà:

- il numero di oggetti trovati (*label_count*);
- un'immagine in cui ad ogni pixel è assegnato l'indice dell'oggetto a cui appartiene (*labels*);
- un array contenente la dimensione (in pixel) e la *bounding box* di ogni oggetto (*stats*);
- un array con le coordinate dei centroidi di ogni oggetto (*centroids*).

La funzione **draw_connected_components(...)** permette di visualizzare ogni candidato con un colore diverso.

In [ ]:
label_count,labels,stats,centroids=#...

print(label_count)

cv2_imshow(draw_connected_components(labels))

Non sempre i candidati individuati in questa fase rappresentano degli oggetti in movimento; in alcuni casi potrebbero essere frutto di rumore che non si è riusciti a rimuovere nella fase di post-processing.

I candidati validi possono essere selezionati sulla base di specifiche caratteristiche quali: area, perimetro, forma, feature colore, ecc..

In questa applicazione i candidati validi sono selezionati sulla base della loro area utilizzando la cella sottostante.

In [ ]:
minimum_valid_area = 100
maximum_valid_area = 15000

detected_bbs=[]
detected_centroids=[]
for i in range(label_count):
    if stats[i,cv2.CC_STAT_AREA]>=minimum_valid_area and stats[i,cv2.CC_STAT_AREA]<=maximum_valid_area:
        detected_bbs.append([(stats[i,cv2.CC_STAT_LEFT],stats[i,cv2.CC_STAT_TOP]),(stats[i,cv2.CC_STAT_LEFT]+stats[i,cv2.CC_STAT_WIDTH],stats[i,cv2.CC_STAT_TOP]+stats[i,cv2.CC_STAT_HEIGHT])])
        detected_centroids.append(centroids[i].astype(int))

frame_with_objects=draw_detected_objects(frame,detected_bbs,detected_centroids,[left_side_contour,right_side_contour])

cv2_imshow(frame_with_objects)

### **Funzione detect_objects**
Completare la funzione sottostante che esegue tutti i passi della fase 2 dati in input:
- il frame corrente (*frame*);
- la maschera binaria della regione di interesse (*roi_mask*);
- l'istanza della classe **BackgroundSubtractorMOG2** con cui è stato stimato il backgorund iniziale nella fase 1 (*bg_subtractor*);
- la soglia di binarizzazione della maschera di foreground (*bin_thr*);
- la dimensione minima di un oggetto valido (*minimum_valid_area*);
- la dimensione massima di un oggetto valido (*maximum_valid_area*);
- il kernel da utilizzare con gli operatori morfologici (*morph_kernel*).  

Seguendo le operazioni svolte precedentemente si dovrà:
1. segmentare il foreground tramite il metodo **apply(...)**;
2. segmentare la maschera di foreground per rimuovere le ombre tramite la funzione **threshold(...)**;
3. intersecare le maschere binarie di foreground e della regione di interesse utilizzando l'operatore logico AND tramite la funzione **bitwise_and(...)**;
4. applicare gli operatori morfologici di chiudura, apertura e dilatazione tramite la funzione **morphologyEx(...)**;
5. richiamare la funzione **connectedComponentsWithStats(...)** per l'individuazione dei candidati tramite l'etichettatura delle componenti connesse.

In [ ]:
def detect_objects(frame,roi_mask,bg_subtractor,bin_thr,minimum_valid_area,maximum_valid_area,morph_kernel=None):
  #1
  fg=#...

  #2
  _,fg=#...

  #3
  fg=#...

  #4
  if morph_kernel is not None:
    fg=#...
    fg=#...
    fg=#...

  #5
  label_count,labels,stats,centroids=#...

  detected_bbs=[]
  detected_centroids=[]
  for i in range(label_count):
    if stats[i,cv2.CC_STAT_AREA]>=minimum_valid_area and stats[i,cv2.CC_STAT_AREA]<=maximum_valid_area:
      detected_bbs.append([(stats[i,cv2.CC_STAT_LEFT],stats[i,cv2.CC_STAT_TOP]),(stats[i,cv2.CC_STAT_LEFT]+stats[i,cv2.CC_STAT_WIDTH],stats[i,cv2.CC_STAT_TOP]+stats[i,cv2.CC_STAT_HEIGHT])])
      detected_centroids.append(centroids[i].astype(int))

  return detected_bbs,detected_centroids,fg

### **Individuazione sulla sequenza video**
Completare il codice sottostante per individuare la posizione degli oggetti in movimento all'interno della sequenza video. In particolare:

1. leggere un nuovo frame;
2. ridimensionare il frame in base al *resize_factor*;
3. utilizzare la funzione **detect_object(...)** appena implementata per individuare la posizione degli oggetti in movimento all'interno del frame corrente.

In [ ]:
detection_frame_count=500

video_capture.set(cv2.CAP_PROP_POS_FRAMES, train_frame_count)

video_detected_objects=[]
fg_detected_objects=[]
frame_count=0
while frame_count<detection_frame_count:
  #1
  ret ,frame=#...

  if ret == False:
    break

  frame_count+=1

  #2
  if resize_factor!=1:
    frame=#...

  #3
  detected_bbs,detected_centroids,fg=#...

  frame_with_objects=draw_detected_objects(frame,detected_bbs,detected_centroids,[left_side_contour,right_side_contour])
  fg_with_objects=draw_detected_objects(cv2.cvtColor(fg, cv2.COLOR_GRAY2BGR),detected_bbs,detected_centroids,[left_side_contour,right_side_contour])

  video_detected_objects.append(frame_with_objects)
  fg_detected_objects.append(fg_with_objects)

Il codice seguente crea un video in cui è riportato il risultato del codice appena eseguito.

In [ ]:
video_detection_both=[]
for i in range(len(video_detected_objects)):
  video_detection_both.append(cv2.cvtColor(np.concatenate((video_detected_objects[i], fg_detected_objects[i]), axis=1),cv2.COLOR_BGR2RGB))

vide_file_name=create_mp4_video_from_frames(video_detection_both,30)

mp4 = open(vide_file_name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(""" <video controls><source src="%s" type="video/mp4"></video>""" % data_url)

## **Fase 3**
L'obiettivo della fase 3 è, dati due frame consecutivi su cui sono stati individuati gli oggetti in movimento (fase 2), abbinare gli oggetti individuati nel frame corrente con quelli individuati nel frame precedente.

Il codice contenuto nella cella sottostante:
1. legge due frame consecutivi dalla sequenza video;
2. ridimensiona i frame in base al *resize_factor*;
3. individua gli oggetti in movimento su entrambi i frame utilizzando la funzione **detect_objects(...)** implementata nella fase 2.

In [ ]:
video_capture.set(cv2.CAP_PROP_POS_FRAMES, train_frame_count)

#1
_,frame_a = video_capture.read()
_,frame_b = video_capture.read()

#2
if resize_factor!=1:
  frame_a=cv2.resize(frame_a,(resized_width,resized_height),cv2.INTER_LINEAR)
  frame_b=cv2.resize(frame_b,(resized_width,resized_height),cv2.INTER_LINEAR)

#3
detected_bbs_a,detected_centroids_a,_=detect_objects(frame_a,roi_mask,bg_subtractor,bin_thr,minimum_valid_area,maximum_valid_area,morph_kernel)
detected_bbs_b,detected_centroids_b,_=detect_objects(frame_b,roi_mask,bg_subtractor,bin_thr,minimum_valid_area,maximum_valid_area,morph_kernel)

frame_a_with_objects=draw_detected_objects(frame_a,detected_bbs_a,detected_centroids_a,[left_side_contour,right_side_contour],(255,0,0))
frame_b_with_objects=draw_detected_objects(frame_b,detected_bbs_b,detected_centroids_b,[left_side_contour,right_side_contour],(255,0,0))

cv2_imshow(np.concatenate((frame_a_with_objects, frame_b_with_objects), axis=1))

### **Abbinamento oggetti tramite Intersection over Union**
L'obiettivo del codice sottostante è quello di selezionare i soli oggetti del frame corrente (*frame_b*) che erano già presenti nel frame precedente (*frame_a*).

Per farlo, completare la funzione sottostante che implementa il seguente algoritmo:
<BR></BR>
Per ogni *bounding box* contenente un oggetto del frame precedente (*detected_bbs_a*), calcolare l'*Intersection over Union* con tutte le *bounding box* degli oggetti del frame corrente (*detected_bbs_b*) e selezionare l'oggetto del frame corrente che presenta l'*Intersection over Union* massima solamente se questa risulta maggiore della soglia *min_iou*.
<BR></BR>
In particolare si implementino le seguenti operazioni:
1. calcolare l'*Intersection over Union* tra l'*i*-esima *bounding box* del frame precedente e la *j*-esima *bounding box* del frame corrente richiamando la funzione **compute_iou(...)** definita all'inizio dell'esercitazione;
2. se l'*Intersection over Union* calcolata al punto precedente è maggiore della massima trovata fino ad ora (*max_iou*), aggiornare *max_iou* e *max_iou_idx*;
3. se l'*Intersection over Union* massima è maggiore della soglia *min_iou*, aggiungere la *bounding box* del frame corrente in posizione *max_iou_idx* alla lista delle *bounding box* abbinate *tracked_bbs*.

In [ ]:
def track_objects(prev_tracked_bbs,prev_tracks,curr_detected_bbs,curr_detected_centroids,min_iou):
  tracked_bbs=[]
  tracks=[]
  curr_tracked_objects=set()
  for i in range(len(prev_tracked_bbs)):
    max_iou=0
    max_iou_idx=None
    for j in range(len(curr_detected_bbs)):
      if j not in curr_tracked_objects:
        #1
        iou=#...
        #2
        if iou>max_iou:
          max_iou=#...
          max_iou_idx=#...

    if max_iou_idx is not None and max_iou>min_iou:
      #3
      tracked_bbs.append(#...)
      prev_track=prev_tracks[i]
      prev_track.append(curr_detected_centroids[max_iou_idx])
      tracks.append(prev_track)
      curr_tracked_objects.add(max_iou_idx)

  return tracked_bbs,tracks,curr_tracked_objects

La cella seguente esegue la funzione **track_object(...)** appena implementata e ne visualizza il risultato.

In [ ]:
min_iou = 0.3

cc=[list([c]) for c in detected_centroids_a]

tracked_bbs,tracks,curr_tracked_objects=track_objects(detected_bbs_a,cc,detected_bbs_b,detected_centroids_b,min_iou)

frame_with_tracked_objects=draw_tracked_objects(frame_b,tracked_bbs,tracks,[left_side_contour,right_side_contour],(0,0,255))

cv2_imshow(np.concatenate((frame_a_with_objects,frame_b_with_objects, frame_with_tracked_objects), axis=1))

### **Inserimento nuovi oggetti**
Può succedere che nel frame corrente siano presenti oggetti appena entrati in scena (oppure non rilevati nel frame precedente) che vengono scartati dalla funzione **track_objects(...)** perchè non hanno un oggetto associato nel frame precedente.

Perciò è opportuno aggiungere agli output ottenuti tutti questi nuovi oggetti così da porterne tracciare il movimento nei frame successivi.

Il codice sottostante aggiunge i nuovi oggetti all'elenco degli oggetti tracciati e visualizza a video il risultato.

In [ ]:
tracked_colors=[(0,0,255)] *len(tracked_bbs)
tracked_colors=tracked_colors+[(255,0,0) for i in range(len(detected_bbs_b)) if i not in curr_tracked_objects]

tracked_bbs=tracked_bbs+[detected_bbs_b[i] for i in range(len(detected_bbs_b)) if i not in curr_tracked_objects]
tracks=tracks+[[detected_centroids_b[i]] for i in range(len(detected_bbs_b)) if i not in curr_tracked_objects]

frame_with_tracked_and_new_objects=draw_tracked_objects(frame_b,tracked_bbs,tracks,[left_side_contour,right_side_contour],tracked_colors=tracked_colors)

cv2_imshow(frame_with_tracked_and_new_objects)

### **Tracciamento sulla sequenza video**
Completare il codice sottostante per tracciare gli oggetti in movimento all'interno della sequenza video. In particolare:

1. leggere un nuovo frame;
2. ridimensionare il frame in base al *resize_factor*;
3. utilizzare la funzione **detect_object(...)** per individuare la posizione degli oggetti in movimento all'interno del frame corrente;
4. utilizzare la funzione **track_objects(...)** per abbinare gli oggetti in movimento individuati nel frame corrente con gli oggetti tracciati dai frame precedenti.

In [ ]:
tracking_frame_count=500
min_iou=0.3

video_capture.set(cv2.CAP_PROP_POS_FRAMES, train_frame_count)

video_tracked_objects=[]
fg_tracked_objects=[]
prev_tracked_bbs=[]
prev_tracks=[]
frame_count=0
while frame_count<tracking_frame_count:
  #1
  ret ,frame=#...

  if ret == False:
    break

  frame_count+=1

  #2
  if resize_factor!=1:
    frame=#...

  #3
  detected_bbs,detected_centroids,fg=#...

  #4
  tracked_bbs,tracks,curr_tracked_objects=#...

  prev_tracked_bbs=tracked_bbs+[detected_bbs[i] for i in range(len(detected_bbs)) if i not in curr_tracked_objects]
  prev_tracks=tracks+[[detected_centroids[i]] for i in range(len(detected_bbs)) if i not in curr_tracked_objects]

  frame_with_tracked_objects=draw_tracked_objects(frame,tracked_bbs,tracks,[left_side_contour,right_side_contour])
  fg_with_tracked_objects=draw_tracked_objects(cv2.cvtColor(fg, cv2.COLOR_GRAY2BGR),tracked_bbs,tracks,[left_side_contour,right_side_contour])

  video_tracked_objects.append(frame_with_tracked_objects)
  fg_tracked_objects.append(fg_with_tracked_objects)

Il codice seguente crea un video in cui è riportato il risultato del codice appena eseguito.

In [ ]:
video_track_both=[]
for i in range(len(video_tracked_objects)):
  video_track_both.append(cv2.cvtColor(np.concatenate((video_tracked_objects[i], fg_tracked_objects[i]), axis=1),cv2.COLOR_BGR2RGB))

vide_file_name=create_mp4_video_from_frames(video_track_both,30)

mp4 = open(vide_file_name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(""" <video controls><source src="%s" type="video/mp4"></video>""" % data_url)

## Esercizio
Testare l'applicazione appena implementata sul video "TownCentreXVID.avi".